## ANALYSE DE SENSIBILITE DU MODEL HYDRO R-NIAYES 2040

In [32]:
create_folder=function(x,path=getwd()){
    
        for (f in x){
            
            folder<-file.path(path, f)

        if (!dir.exists(folder)){
            dir.create(folder)
            print(paste(f,"has been created!"))
        } else {
        print(paste(folder,"already exists!"))}
        }
}

In [33]:
rcsv_sep=function(file_path){
    L <- readLines(file_path, n = 1)
    if (grepl(";", L)) read.csv2(file_path) else read.csv(file_path)
    }

In [34]:
normaliz_zero<-function(x){
    x<-na.omit(x)
    min<-min(x)
    max<-max(x)
    
    val<-lapply(x,function(y) (y-min)/(max-min))
return(val)
}

### Méthode de Morris

### 1-  LE PLAN DE MORRIS

### a-)  Plan d'expérience

- les paramètres d'entrée

In [35]:
#### la table des RMSE
rmse<-rcsv_sep('C:/Users/kouno/Desktop/Programme_Crésus/Calibration Model Hydro/output/tableau_rsme.csv')[-1]
names(rmse)[4:7]<-c("varperm","varstock","nbiterations","consourb")
head(rmse,3)
rmse[3:7]<-data.frame(lapply(rmse[3:7],as.numeric))

,X,simul,resolution,varperm,varstock,nbiterations,consourb,Y_mean,Y_median,rmse_moyen,avg_rmse_moyen,pond_avg_rmse,emad,rmsle,sum
,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Niayes,simulation 1,1000.0,-0.001,-30.0,23,0.03,3.452793,0.7838115,0.5943249,0.7671572,0.6455347,0.3866388,0.1018249,3.279292
2,Niayes,simulation 1,200.0,-0.001,-30.0,43,0.03,2.729792,1.5698361,0.7485651,0.6694341,0.7485651,0.3332178,0.2084089,4.278027
3,Niayes,simulation 1,300.0,-0.001,-30.0,23,0.03,2.423022,0.7665675,0.5894097,0.5726000,0.5894097,0.2999395,0.1132734,2.931200


- valeurs des paramètres

In [36]:
parList<-sapply(rmse[3:7],unique)

In [37]:
X<- data.frame(lapply(parList, function(x) {
  x <- unlist(x)
  length(x) <- max(lengths(parList))
  return(x)
}))
X

resolution,varperm,varstock,nbiterations,consourb
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1000,-1e-03,-30,23,0.03
200,0e+00,-60,43,0.06
300,1e-03,0,3,NA
400,5e-04,30,NA,NA
500,-5e-04,60,NA,NA
600,NA,NA,NA,NA
700,NA,NA,NA,NA
800,NA,NA,NA,NA
900,NA,NA,NA,NA


- les deltas des parramètres

In [38]:
D<-X[which(X[1]=="R"),]
D["delta",]<-c(100,max(na.omit(X[2]))/2,
               max(na.omit(X[3]))/2,20,0.03)
D

,resolution,varperm,varstock,nbiterations,consourb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
delta,100,5e-04,30,20,0.03


- ecart "delta" normalisé entre 0 et 1

In [39]:
D_norm<-X[which(X[1]=="R"),]
D_norm["norm_delta",]<-c(0.125,0.25,0.25,0.5,1)
D_norm

,resolution,varperm,varstock,nbiterations,consourb
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
norm_delta,0.125,0.25,0.25,0.5,1


#### b-) Plan One At Time (OAT)

In [40]:
Y<-rmse[3:10]
head(Y,3)

,resolution,varperm,varstock,nbiterations,consourb,Y_mean,Y_median,rmse_moyen
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1000,-0.001,-30,23,0.03,3.452793,0.7838115,0.5943249
2,200,-0.001,-30,43,0.03,2.729792,1.5698361,0.7485651
3,300,-0.001,-30,23,0.03,2.423022,0.7665675,0.5894097


##### premier plan OAT

- trier au hasard l'état initial des paramètre du plan OAT

In [41]:
par1<-Y[sample(length(na.omit(Y[,1])),1),1]
par2<-Y[sample(length(na.omit(Y[,2])),1),2]
par3<-Y[sample(length(na.omit(Y[,3])),1),3]
par4<-Y[sample(length(na.omit(Y[,4])),1),4]
par5<-Y[sample(length(na.omit(Y[,5])),1),5]
Mylist<-c(par1,par2,par3,par4,par5)

- réponse moyenne du model au set initial de paramètres

###### run 0

In [42]:
y0<-round(Y[which(Y[,1]==Mylist[1] & 
                 Y[,2]==Mylist[2] & 
                 Y[,3]==Mylist[3] &
                 Y[,4]==Mylist[4] &
                 Y[,5]==Mylist[5]),
                 "Y_mean"],3)

###### tirage aléatoire et modification de l'entrée tirée

In [43]:
ci<-sample(length(names(X)),1)
chi<-Mylist[ci]
if (chi==min(na.omit(X[ci]))){
    e<-D[[ci]]
    chi<-chi+e
}
if (chi==max(na.omit(X[ci]))){
    e<-D[[ci]]
    chi<-chi-e
}
if (chi!=min(na.omit(X[ci])) & chi!=max(na.omit(X[ci])) ){
    plus_moins_di<-c(D[[ci]],-D[[ci]])
    e<-plus_moins_di[sample(1:2,1)]
    chi<-chi+e
}
Mylist[ci]<-chi

###### run 1

In [44]:
y1<-round(Y[which(Y[,1]==Mylist[1] & 
                 Y[,2]==Mylist[2] & 
                 Y[,3]==Mylist[3] &
                 Y[,4]==Mylist[4] &
                 Y[,5]==Mylist[5]),
                 "Y_mean"],3)

###### effet élémentaire E1

In [50]:
Mylist

[1]  3e+02 -1e-03 -6e+01  3e+00  3e-02

- Calcul d’indices de sensibilité

In [44]:
list_sensitiv

[1]  5.000117  6.880316  6.063959  8.360279 12.411683  4.190257

### L’indice de Bauer-Hamby index

#### Indice de sensibilité basés sur une décomposition de la variance